## 1. Data Extraction & Transformation
### Complete ETL Pipeline for Online Retail Data
### By: Chiadika Elue

In [14]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [15]:
# Set up plotting for VS Code
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style('whitegrid')

In [16]:

# 1. DATA EXTRACTION
print("\n1.  DATA EXTRACTION")
# Load dataset
df = pd.read_csv(r'C:\Users\User\Desktop\DataMining_GroupProject_ChiadikaKeyshiaNelisa\data\raw\online_retail_II.csv')
print(f"Data loaded: {df.shape[0]:,} rows, {df.shape[1]} columns")

# Display initial info
print("\n Initial Data Overview:")
print(f"Columns: {list(df.columns)}")
df.head()


1.  DATA EXTRACTION
Data loaded: 1,067,371 rows, 8 columns

 Initial Data Overview:
Columns: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [17]:
# 2. DATA QUALITY ASSESSMENT
print("DATA QUALITY ASSESSMENT")

print("Missing Values:")
missing_data = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df)) * 100
})
print(missing_data)

print(f"\nDuplicate rows: {df.duplicated().sum()}")

print("\nData Types:")
print(df.dtypes)

# Check for negative quantities and prices
negative_qty = df[df['Quantity'] < 0].shape[0]
negative_price = df[df['Price'] < 0].shape[0]
print(f"\nNegative quantities: {negative_qty}")
print(f"Negative prices: {negative_price}")


DATA QUALITY ASSESSMENT
Missing Values:
             Missing Count  Missing %
Invoice                  0   0.000000
StockCode                0   0.000000
Description           4382   0.410541
Quantity                 0   0.000000
InvoiceDate              0   0.000000
Price                    0   0.000000
Customer ID         243007  22.766873
Country                  0   0.000000

Duplicate rows: 34335

Data Types:
Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

Negative quantities: 22950
Negative prices: 5


In [18]:
# 3. DATA CLEANING
print("3.  DATA CLEANING")

# Store original row count
original_rows = len(df)
df_clean = df.copy()

# Handle missing values
print("Handling missing values...")
df_clean = df_clean.dropna(subset=['Customer ID'])
df_clean['Description'] = df_clean['Description'].fillna('Unknown')

# Convert InvoiceDate to datetime
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# Remove cancelled orders (negative quantities)
df_clean = df_clean[df_clean['Quantity'] > 0]
df_clean = df_clean[df_clean['Price'] > 0]

# Remove duplicates
df_clean = df_clean.drop_duplicates()

print(f" Data cleaning completed:")
print(f"   - Original rows: {original_rows:,}")
print(f"   - Cleaned rows: {len(df_clean):,}")
print(f"   - Rows removed: {original_rows - len(df_clean):,}")

# Verify cleaning
print("\n Data Quality After Cleaning:")
print(f"Missing values: {df_clean.isnull().sum().sum()}")
print(f"Negative quantities: {len(df_clean[df_clean['Quantity'] < 0])}")
print(f"Duplicate rows: {df_clean.duplicated().sum()}")

3.  DATA CLEANING
Handling missing values...
 Data cleaning completed:
   - Original rows: 1,067,371
   - Cleaned rows: 779,425
   - Rows removed: 287,946

 Data Quality After Cleaning:
Missing values: 0
Negative quantities: 0
Duplicate rows: 0


In [19]:
# 4. FEATURE ENGINEERING
print("4.  FEATURE ENGINEERING")

# Calculate total amount
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['Price']

# Extract date components
df_clean['InvoiceYear'] = df_clean['InvoiceDate'].dt.year
df_clean['InvoiceMonth'] = df_clean['InvoiceDate'].dt.month
df_clean['InvoiceDay'] = df_clean['InvoiceDate'].dt.day
df_clean['InvoiceHour'] = df_clean['InvoiceDate'].dt.hour
df_clean['InvoiceDayOfWeek'] = df_clean['InvoiceDate'].dt.day_name()

# Create month-year identifier
df_clean['YearMonth'] = df_clean['InvoiceDate'].dt.to_period('M')

print(" Features created:")
print("   - TotalAmount: Quantity * Price")
print("   - Time-based features (Year, Month, Day, Hour, DayOfWeek)")
print("   - YearMonth for time series analysis")

df_clean[['InvoiceDate', 'TotalAmount', 'InvoiceYear', 'InvoiceMonth', 'InvoiceDayOfWeek']].head()

4.  FEATURE ENGINEERING
 Features created:
   - TotalAmount: Quantity * Price
   - Time-based features (Year, Month, Day, Hour, DayOfWeek)
   - YearMonth for time series analysis


,InvoiceDate,TotalAmount,InvoiceYear,InvoiceMonth,InvoiceDayOfWeek
0,2009-12-01 07:45:00,83.4,2009,12,Tuesday
1,2009-12-01 07:45:00,81.0,2009,12,Tuesday
2,2009-12-01 07:45:00,81.0,2009,12,Tuesday
3,2009-12-01 07:45:00,100.8,2009,12,Tuesday
4,2009-12-01 07:45:00,30.0,2009,12,Tuesday


In [20]:
# 5. RFM METRICS CALCULATION
print("5.  RFM METRICS CALCULATION")

# Set analysis date (day after last invoice)
analysis_date = df_clean['InvoiceDate'].max() + timedelta(days=1)
print(f"Analysis date for RFM: {analysis_date}")

# Calculate RFM metrics per customer
rfm_df = df_clean.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (analysis_date - x.max()).days,  # Recency
    'Invoice': 'nunique',  # Frequency
    'TotalAmount': 'sum'   # Monetary
}).reset_index()

# Rename columns
rfm_df.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

print(" RFM metrics calculated:")
print(f"   - Customers analyzed: {len(rfm_df):,}")
print(f"   - Recency range: {rfm_df['Recency'].min()} to {rfm_df['Recency'].max()} days")
print(f"   - Frequency range: {rfm_df['Frequency'].min()} to {rfm_df['Frequency'].max()} purchases")
print(f"   - Monetary range: ${rfm_df['Monetary'].min():.2f} to ${rfm_df['Monetary'].max():.2f}")

rfm_df.head()

5.  RFM METRICS CALCULATION
Analysis date for RFM: 2011-12-10 12:50:00
 RFM metrics calculated:
   - Customers analyzed: 5,878
   - Recency range: 1 to 739 days
   - Frequency range: 1 to 398 purchases
   - Monetary range: $2.95 to $580987.04


,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,12,77556.46
1,12347.0,2,8,4921.53
2,12348.0,75,5,2019.40
3,12349.0,19,4,4428.69
4,12350.0,310,1,334.40


In [21]:
# 6. DATA VALIDATION & SAVING
print("6.  DATA VALIDATION & SAVING")

# Validate data quality
print("Data Validation Checks:")
print(f" No negative quantities: {len(df_clean[df_clean['Quantity'] < 0]) == 0}")
print(f" No negative prices: {len(df_clean[df_clean['Price'] < 0]) == 0}")
print(f" No missing Customer IDs: {df_clean['Customer ID'].isnull().sum() == 0}")
print(f" Valid date range: {df_clean['InvoiceDate'].min()} to {df_clean['InvoiceDate'].max()}")

# Save cleaned data
df_clean.to_csv('C:/Users/User/Desktop/DataMining_GroupProject_ChiadikaKeyshiaNelisa/data/transformed/cleaned_retail_data.csv', index=False)
rfm_df.to_csv('../data/transformed/customer_rfm_data.csv', index=False)

print("\n Data saved:")
print(" /data/transformed/cleaned_retail_data.csv")
print(" /data/transformed/customer_rfm_data.csv")

6.  DATA VALIDATION & SAVING
Data Validation Checks:
 No negative quantities: True
 No negative prices: True
 No missing Customer IDs: True
 Valid date range: 2009-12-01 07:45:00 to 2011-12-09 12:50:00

 Data saved:
 /data/transformed/cleaned_retail_data.csv
 /data/transformed/customer_rfm_data.csv


In [22]:

# 7. FINAL SUMMARY
print("7. ETL PIPELINE SUMMARY")

summary_stats = {
    'Original Records': original_rows,
    'Cleaned Records': len(df_clean),
    'Data Retention Rate': f"{(len(df_clean)/original_rows*100):.1f}%",
    'Total Customers': len(rfm_df),
    'Total Revenue': f"${df_clean['TotalAmount'].sum():,.2f}",
    'Date Range': f"{df_clean['InvoiceDate'].min().strftime('%Y-%m-%d')} to {df_clean['InvoiceDate'].max().strftime('%Y-%m-%d')}",
    'Countries': df_clean['Country'].nunique()
}

print("\n Pipeline Summary:")
for key, value in summary_stats.items():
    print(f"   {key}: {value}")

print("\n ETL Pipeline Completed Successfully")


7. ETL PIPELINE SUMMARY

 Pipeline Summary:
   Original Records: 1067371
   Cleaned Records: 779425
   Data Retention Rate: 73.0%
   Total Customers: 5878
   Total Revenue: $17,374,804.27
   Date Range: 2009-12-01 to 2011-12-09
   Countries: 41

 ETL Pipeline Completed Successfully
